In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
import collections
from tqdm import tqdm
from xqdm import xqdm

# tvr=0.05인것들만으로... 0.01인거는 과적합해보임

In [2]:
def softmax(x):
    ex = np.exp(x)
    total = np.sum(ex)
    return ex / total

def preprocessing(csv, score):
    data = {'id': [], 'vals': [], 'score': score}
    data.update({str(i): [] for i in range(10)})
    for _, row in csv.iterrows():
        data['id'].append(row[0])
        vals = softmax(row[1:].to_numpy(np.float32)) * score
        data['vals'].append(vals)
    return data

def __preprocessing(args):
    return preprocessing(*args)

def get_score(filename):
    score = re.search('submission-raw-L\\d{4}-(0\\.\\d+)\\.csv', str(filename))
    score = score.groups()[0]
    score = float(score)
    #return np.exp(1 / score)
    return 1 / score

In [8]:
submission_files = list(Path('submissions/prep1').glob('submission-raw-*.csv'))

In [4]:
items = [(pd.read_csv(f), get_score(f)) for f in submission_files]

In [5]:
datas = xqdm(__preprocessing, items)

100%|██████████| 18/18 [00:14<00:00,  1.26it/s]


In [6]:
datas[0]['vals'][0], datas[0]['score']

(array([3.7052756e-04, 5.8356876e-05, 3.3303875e-02, 2.6621879e-04,
        2.2623045e-03, 1.2195480e-02, 3.7489227e+01, 1.5722773e-05,
        1.9951448e-02, 1.5846254e-05], dtype=float32),
 37.557663948333044)

In [9]:
submission_files[0]

PosixPath('submissions/prep1/submission-raw-L3002-0.18245945125818253.csv')

In [12]:
for data in datas:
    data['vals'] = np.stack(data['vals'])

In [13]:
datas[0]['vals'].shape

(20480, 10)

In [14]:
vals = [data['vals'] for data in datas]

In [15]:
vals = np.stack(vals)

In [16]:
vals.shape

(18, 20480, 10)

In [17]:
valsum = np.sum(vals, axis=0)

In [18]:
valsum.shape

(20480, 10)

In [19]:
valarg = np.argmax(valsum, axis=1)

In [20]:
valarg.shape

(20480,)

In [27]:
newcsv = {'id': list(map(int, datas[0]['id'])), 'digit': list(valarg)}

In [28]:
newcsv = pd.DataFrame(newcsv)

In [29]:
newcsv

,id,digit
0,2049,6
1,2050,9
2,2051,8
3,2052,0
4,2053,3
...,...,...
20475,22524,4
20476,22525,1
20477,22526,6
20478,22527,3


In [30]:
newcsv.to_csv('./submissions/prep1/ensemble-all.csv', index=False)